In [78]:
import pandas as pd
import numpy as np
from datetime import datetime
import pyxirr
import yfinance as yf

In [54]:
df = pd.read_csv('final_dataset.csv')

In [ ]:
as_of_date = ""
symbols = df['Symbol'].dropna().unique()

price_data = []
for symbol in symbols:
    try:
        ticker = yf.Ticker(symbol)
        hist = ticker.history(start="2025-07-17", end="2025-07-18")
        if not hist.empty:
            price = hist['Close'][0]
        else:
            price = None
    except Exception:
        price = None
    price_data.append({'Symbol': symbol, 'Price': price, 'Date':"2025-07-17"})

prices_df = pd.DataFrame(price_data)
prices_df

In [66]:
# prices_df.to_csv('StockPrice_17July/symbol_prices_2025-07-17.csv', index=False)
prices_df = pd.read_csv('StockPrice_17July/symbol_prices_2025-07-17.csv')

,Symbol,Price,Date
0,NET,191.250000,2025-07-17
1,NVDA,173.000000,2025-07-17
2,SPY,628.039978,2025-07-17
3,MSFT,511.700012,2025-07-17
4,AMZN,223.880005,2025-07-17
5,GOOG,184.699997,2025-07-17
6,GPRO,0.784000,2025-07-17
7,SE,167.080002,2025-07-17
8,TSLA,319.410004,2025-07-17
9,MARA,19.969999,2025-07-17


In [77]:
current_value_date = pd.to_datetime("2025-07-17")
xirr_results = {}
for symbol in df['Symbol'].unique():
    holding = df[df['Symbol'] == symbol].copy()
    holding['Date/Time'] = holding['Date/Time'].astype(str).str.strip()
    holding['Date/Time'] = pd.to_datetime(holding['Date/Time'], errors='coerce')
    holding = holding.dropna(subset=['Date/Time'])
    cashflows = holding['Proceeds'].tolist()
    dates = holding['Date/Time'].tolist()
    qty = holding['Quantity'].sum()
    if qty != 0:
        price_row = prices_df[prices_df['Symbol'] == symbol]
        if not price_row.empty and pd.notnull(price_row['Price'].iloc[0]):
            current_price = price_row['Price'].iloc[0]
        current_value = qty * current_price
        cashflows.append(current_value)
        dates.append(current_value_date)
    if not all(isinstance(d, pd.Timestamp) for d in dates):
        xirr_results[symbol] = "Error: Invalid date in cashflows"
    try:
        result = pyxirr.xirr(dates, cashflows)
        xirr_results[symbol] = result
    except Exception as e:
        xirr_results[symbol] = f"Error: {e}"

for symbol, result in xirr_results.items():
    if isinstance(result, float):
        print(f"{symbol}: {result:.2%}")
    else:
        print(f"{symbol}: {result}")

NET: 46.21%
NVDA: 88.21%
SPY: 23.14%
MSFT: 21.78%
AMZN: 27.42%
GOOG: 19.04%
GPRO: -67.51%
SE: 99.83%
TSLA: 38.77%
MARA: 19.41%
NU: -31.01%
IBIT: 86.23%
C6L: Error: negative and positive payments are required
AAPL: 8.36%
AXP: 29.73%
CMG: -13.67%
GOOGL: -9.52%
ORCL: 85.00%


In [79]:
xirr_results

{'NET': 0.4620576195346279,
 'NVDA': 0.8821308625654549,
 'SPY': 0.2313926598277846,
 'MSFT': 0.21776609350508896,
 'AMZN': 0.27418450806194944,
 'GOOG': 0.19044747091956762,
 'GPRO': -0.6751396228176587,
 'SE': 0.9983108843303778,
 'TSLA': 0.3877391727026749,
 'MARA': 0.19407508924582173,
 'NU': -0.31007941416712437,
 'IBIT': 0.8623174327186169,
 'C6L': 'Error: negative and positive payments are required',
 'AAPL': 0.08364357849903294,
 'AXP': 0.29726316688645693,
 'CMG': -0.1366949546609154,
 'GOOGL': -0.09523216647557807,
 'ORCL': 0.8499838048005866}